In [127]:
import requests
r = requests.get("https://projecteuler.net/project/resources/p081_matrix.txt")
def get_final_matrix(r):
    result = []
    temp1 = r.text.split("\n")
    for i in temp1:
        temp2 = i.split(",")
        temp2_int = [i for i in temp2]
        result.append(temp2_int)
    return result

def convert_to_int(list_of_str):
    result = []
    for ele in list_of_str:
        result.append(int(ele))
    return result



final_matrix = list(map(convert_to_int,get_final_matrix(r)[:-1]))
final_matrix_arr = np.array(final_matrix,dtype='int64')

In [128]:
import numpy as np
test_matrix = [[131,673,234,103,18],[201,96,342,965,150],[630,803,746,422,111],[537,699,497,121,956],[805,732,524,37,331]]
test_mat_np = np.array(test_matrix)

In [129]:
test_mat_np

array([[131, 673, 234, 103,  18],
       [201,  96, 342, 965, 150],
       [630, 803, 746, 422, 111],
       [537, 699, 497, 121, 956],
       [805, 732, 524,  37, 331]])

In [163]:
# Implemetation of Djikstra algorithm
import re
import sys
class Node(object):
    def __init__(self,name,value,dist_from_source):
        self.name = name
        self.value = value
        self.dist_from_source = dist_from_source
        self.adj_nodes = []
        
    def get_dist_from_source(self):
        return self.dist_from_source
    
    def set_dist_from_source(self, dist):
        self.dist_from_source = dist
        
    def get_name(self):
        return self.name
    
    def get_value(self):
        return self.value
    

class DirectedGraph(object):
    def __init__(self,list_of_nodes,adj_dict,source_node,I_MAX,J_MAX):
        self.list_of_nodes = list_of_nodes
        self.I_MAX = I_MAX
        self.J_MAX = J_MAX
        self.adj_dict = adj_dict
#         self.source_node = list(filter(lambda x:x.get_name() == "X_0_0",list_of_nodes))[0]
        self.source_node = source_node
        self.dest_nodes = list(filter(lambda x:self.get_dest_nodes(x,J_MAX=J_MAX),list_of_nodes))
#         self.visited_nodes = []
        
    def get_dest_nodes(self,x,J_MAX):
        '''Filter function to check if a node is the right most node'''
        pattern = re.compile(r"X_(\d+)_(\d+)")
        search = re.search(pattern,x.get_name())
        i = None
        j = None
        if search:
            i = search.group(1)
            j = search.group(2)
        else:
            return None
        return int(j) == J_MAX-1
        
    def initialize_nodes_list(self):
        '''Initializes distance on source node to zero and the rest to inf'''
        self.source_node.dist_from_source = self.source_node.get_value()
        for node in self.list_of_nodes:
            node.set_dist_from_source = sys.maxsize
        

            
    def pick_closest_node(self,list_of_unvisited_nodes):
        return sorted(list_of_unvisited_nodes,key=lambda x:x.get_dist_from_source())[0]
    
    def update_dist(self,close_node,adj_nodes):
#         print("close node: {}".format(close_node.get_name()))
        for aj in adj_nodes:
#             print("dist_from_source: {}".format(close_node.get_dist_from_source()))
#             print("aj name: {}".format(aj.get_name()))
#             print("Value: {}".format(aj.get_value()))
            temp = close_node.get_dist_from_source() + aj.get_value() 
#             print("Temp sum: {}".format(temp))
            if (temp < aj.get_dist_from_source()):
                aj.dist_from_source = temp
                

    
        
    def run_spt(self):
        '''Run the shortest path to dest node and return the distance value'''
        self.initialize_nodes_list()
        visited_nodes = []
        list_of_unvisited_nodes = self.list_of_nodes[:]
        
        
        while len(visited_nodes) < len(self.list_of_nodes):
            close_node = self.pick_closest_node(list_of_unvisited_nodes)
            visited_nodes.append(close_node)
            adj_nodes = self.adj_dict[close_node.get_name()]          
            self.update_dist(close_node,adj_nodes)
#             print(close_node.get_name())
#             print([(i.get_dist_from_source(),i.get_name()) for i in adj_nodes])  
            list_of_unvisited_nodes.remove(close_node)
        
#         print(self.dest_node.get_name())
        return [x.get_dist_from_source() for x in self.dest_nodes]



In [164]:
import sys
def create_nodes_list(mat_np):
    list_of_nodes = []
    for i in range(len(mat_np)):
        for j in range(len(mat_np[i,:])):
            list_of_nodes.append(Node("X_{}_{}".format(i,j),mat_np[i,j],sys.maxsize))
            
    return list_of_nodes
            
def extract_node_coords(name):
    '''Returns (i,j) tuple'''
    pattern = re.compile(r'X_(\d+)_(\d+)')
    search = pattern.search(name)
    if search:
        digit_str_1 = search.group(1)
        digit_str_2 = search.group(2)
    return (int(digit_str_1),int(digit_str_2))
    
def get_adj_node_names(i,j,I_MAX,J_MAX):
    '''Returns a list of adjacent node_names'''
    adj_node_names = []
    if i+1 < I_MAX:
        adj_node_names.append("X_{}_{}".format(i+1,j))
    if j+1 < J_MAX:
        adj_node_names.append("X_{}_{}".format(i,j+1))
    if i-1 > -1:
        adj_node_names.append("X_{}_{}".format(i-1,j))
    return adj_node_names
            
    

def create_adj_nodes_dict(list_of_nodes,I_MAX,J_MAX):
    '''Creates a dict of node_names mapped to adj nodes'''
    adj_dict = {}
    
    
    for node in list_of_nodes:
        adj_nodes = []
        i,j = extract_node_coords(node.get_name())
        adj_node_names = get_adj_node_names(i,j,I_MAX,J_MAX)
        for an in adj_node_names:
            f = filter(lambda x:x.get_name() == an,list_of_nodes)
            adj_nodes.append(list(f)[0])
        adj_dict[node.get_name()] = adj_nodes

    return adj_dict

In [165]:
test_mat_np = final_matrix_arr
test_list_of_nodes = create_nodes_list(test_mat_np)
adj_dict = create_adj_nodes_dict(test_list_of_nodes,test_mat_np.shape[0],test_mat_np.shape[1])

source_indices = list(range(0,len(test_list_of_nodes),test_mat_np.shape[1]))
# source_nodes = [test_list_of_nodes[ind] for ind in source_indices]

min_per_source = []

for ind in source_indices:
    print(ind)
    dg = DirectedGraph(test_list_of_nodes,adj_dict,test_list_of_nodes[ind],test_mat_np.shape[0],test_mat_np.shape[1])
    min_per_source.append(min(dg.run_spt()))
min(min_per_source)


0
80
160
240
320
400
480
560
640
720
800
880
960
1040
1120
1200
1280
1360
1440
1520
1600
1680
1760
1840
1920
2000
2080
2160
2240
2320
2400
2480
2560
2640
2720
2800
2880
2960
3040
3120
3200
3280
3360
3440
3520
3600
3680
3760
3840
3920
4000
4080
4160
4240
4320
4400
4480
4560
4640
4720
4800
4880
4960
5040
5120
5200
5280
5360
5440
5520
5600
5680
5760
5840
5920
6000
6080
6160
6240
6320


260324